Notebook to test the models used for FL in a traditional approach to check their performance.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torchvision.models import EfficientNet_B0_Weights

batch_size = 1024

# Define transformations
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

# Define the EfficientNet-B0 model
def get_model():
    weights = EfficientNet_B0_Weights.IMAGENET1K_V1
    model = models.efficientnet_b0(weights=weights)
    # Modify classifier head for CIFAR-10 (10 output classes)
    num_ftrs = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(num_ftrs, 10)
    return model

model = get_model()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to evaluate the model on the test set
def evaluate(model, testloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Train and evaluate the model
num_epochs = 20  # You can adjust the number of epochs
for epoch in range(num_epochs):
    running_loss = 0.0
    model.train()
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(trainloader.dataset)
    accuracy = evaluate(model, testloader, device)
    print(f'[{epoch + 1}/{num_epochs}] Test Accuracy: {accuracy:.2f}%,\t loss: {epoch_loss:.4f}')

print('Finished Training')

[1/20] Test Accuracy: 69.36%,	 loss: 1.3011
[2/20] Test Accuracy: 77.20%,	 loss: 0.6734
[3/20] Test Accuracy: 78.65%,	 loss: 0.4603
[4/20] Test Accuracy: 79.49%,	 loss: 0.3249
[5/20] Test Accuracy: 80.24%,	 loss: 0.2329
[6/20] Test Accuracy: 80.06%,	 loss: 0.1738
[7/20] Test Accuracy: 80.60%,	 loss: 0.1481
[8/20] Test Accuracy: 80.76%,	 loss: 0.1256
[9/20] Test Accuracy: 81.20%,	 loss: 0.0998
[10/20] Test Accuracy: 80.64%,	 loss: 0.0900
[11/20] Test Accuracy: 81.06%,	 loss: 0.0816
[12/20] Test Accuracy: 81.53%,	 loss: 0.0743
[13/20] Test Accuracy: 81.57%,	 loss: 0.0559
[14/20] Test Accuracy: 81.81%,	 loss: 0.0569
[15/20] Test Accuracy: 81.73%,	 loss: 0.0522
[16/20] Test Accuracy: 82.16%,	 loss: 0.0479
[17/20] Test Accuracy: 82.59%,	 loss: 0.0474
[18/20] Test Accuracy: 82.12%,	 loss: 0.0621
[19/20] Test Accuracy: 82.11%,	 loss: 0.0511
[20/20] Test Accuracy: 82.33%,	 loss: 0.0384
Finished Training
Final Accuracy of the network on the 10000 test images: 82.33%


In [10]:
def print_model_parameters(net):
    total_params = sum(p.numel() for p in net.parameters())
    trainable_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
    print(f"Total parameters: {total_params:,}")
    print(f"Trainable parameters: {trainable_params:,}")

print_model_parameters(model)

Total parameters: 4,020,358
Trainable parameters: 4,020,358
